In [8]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt

np.set_printoptions(6, suppress=True)

plt.style.use([
    'grid',
    'notebook'
])

In [9]:
A_vertices = []
B_vertices = []

Ts = 2

r = 31
mu = 40
sigma = 55
A2 = np.pi * r**2

Kb = 16

In [10]:
h1_interval = np.linspace(18.83, 45.48, 1000)
h2_interval = np.linspace(12.85, 33, 1000)

Z1 = []
Z2 = []
Z3 = []

c = 0
min_diff = 2.5


def R_34(diff):
    return -0.000071183*diff**2 + 0.002279342*diff + 0.009212421

def q_out(h4):
    return -0.049605*h4**2+10.759176*h4+157.705535


for i in range(len(h1_interval)):
    h1 = h1_interval[i]
    
    for j in range(i, len(h1_interval)): 
        h2 = h2_interval[j]

        diff = h1 - h2

        if diff < min_diff:
            continue

        R34 = 0.73835215*R_34(diff)
        qout = 1.39518809*q_out(h2)

        a3 = (3 * r / 5) * (2.7 * r - ((np.cos(2.5*np.pi*(h1 - 8.) - mu)) / (sigma * np.sqrt(2 * np.pi))) * np.exp(-(((h1 - 8.) - mu)**2) / (2 * sigma ** 2)))

        z1 = 1/R34
        z2 = qout/h2
        z3 = 1/a3

        Z1.append(z1)
        Z2.append(z2)
        Z3.append(z3)


min_z1, max_z1 = min(Z1), max(Z1)
min_z2, max_z2 = min(Z2), max(Z2)
min_z3, max_z3 = min(Z3), max(Z3)

print(min_z1, max_z1)
print(min_z2, max_z2)
print(min_z3, max_z3)

50.97004493299016 93.62094793763693
19.39474208588148 31.244617854110835
0.0006422797019798464 0.0006423903134466694


In [11]:
z1_bounds = np.array([min_z1, max_z1])
z2_bounds = np.array([min_z2, max_z2])
z3_bounds = np.array([min_z3, max_z3])

np.save('./data/z1_bounds.npy', z1_bounds)
np.save('./data/z2_bounds.npy', z2_bounds)
np.save('./data/z3_bounds.npy', z3_bounds)

for z1 in z1_bounds:
    for z2 in z2_bounds:
        for z3 in z3_bounds:
            Ai = np.array([
                [1 - z1*z3, z1*z3],
                [z1/A2, 1 + (-z1-z2)/A2]
            ])

            Bi = np.array([
                [Ts*Kb*z3],
                [.0]
            ])

            print(
                np.abs(np.linalg.eigvals(Ai))
            )

            A_vertices.append(Ai)
            B_vertices.append(Bi)

[0.948001 0.995956]
[0.947995 0.995955]
[0.946346 0.993686]
[0.94634  0.993685]
[0.90657  0.995866]
[0.90656  0.995865]
[0.905065 0.993445]
[0.905055 0.993445]


In [12]:
Z1_, Z2_, Z3_ = sp.symbols('Z1 Z2 Z3')

z1_min, z1_max = z1_bounds
z2_min, z2_max = z2_bounds
z3_min, z3_max = z3_bounds

M1 = (z1_max - Z1_) / (z1_max - z1_min)
N1 = (z2_max - Z2_) / (z2_max - z2_min)
P1 = (z3_max - Z3_) / (z3_max - z3_min)

M2 = 1 - M1
N2 = 1 - N1
P2 = 1 - P1

In [13]:
print(' ')
print(M1)
print(N1)
print(P1)
print(' ')
print(M2)
print(N2)
print(P2)

 
2.19505195300172 - 0.0234461624386019*Z1
2.6367042545611 - 0.0843890703631759*Z2
5807.62855694212 - 9040654.00018561*Z3
 
0.0234461624386019*Z1 - 1.19505195300172
0.0843890703631759*Z2 - 1.6367042545611
9040654.00018561*Z3 - 5806.62855694212


In [14]:
from scipy.io import savemat

for i in range(len(A_vertices)):
    A = A_vertices[i]
    B = B_vertices[i]

    mat = {
        'A': A,
        'B': B
    }

    savemat(f'./data/vertices/vertex_{i}.mat', mat)

    np.save(f'./data/vertices/A_{i}', A)
    np.save(f'./data/vertices/B_{i}', B)